# **Loan Default Prediction**

This notebook demonstrates training a **PyTorch neural network** to predict whether a loan will default.

## **Notebook Outline**
1. **Train-Test Split & Imbalance Handling**
2. **PyTorch Model Building & Training**
3. **Evaluation & Threshold Tuning**

We'll see both **Markdown explanations** (like this one) and **Code cells** demonstrating each step.

---

In [1]:
# (Cell 1) 1. Imports & Setup
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (confusion_matrix, precision_score, recall_score, 
                             f1_score, roc_auc_score, classification_report)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import RandomOverSampler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

plt.style.use('seaborn')  # optional aesthetics
print("Setup complete.")

Setup complete.


<ipython-input-1-7137d4bfe437>:23: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')  # optional aesthetics


## **Cleaned Data Loading For Traning**
We load the "cleaned_train_data.csv" file that we saved after EDA analysis for training.

In [3]:
df = pd.read_csv("cleaned_train_data.csv")


## **6. Train-Test Split & Imbalance Handling**

We'll do a stratified split (to preserve the ~7% minority class). Then we optionally oversample the minority class using `RandomOverSampler` from `imblearn`.

In [4]:
# (Cell 5) Train-Val Split, Scale, and Oversample

y = df['bad_flag'].astype(int)
X = df.drop(columns=['bad_flag'])

# Stratified split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train shape:", X_train.shape, "Val shape:", X_val.shape)
print("Positive class in train:", (y_train==1).mean())
print("Positive class in val:  ", (y_val==1).mean())

# Scale numeric features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)

# Convert back to DataFrame (optional)
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_val   = pd.DataFrame(X_val_scaled,   columns=X_val.columns,   index=X_val.index)

# OverSampling
ros = RandomOverSampler(random_state=42, sampling_strategy=0.15)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)
print("After oversampling:", X_train_ros.shape)
print("Positive class proportion:", (y_train_ros==1).mean())

Train shape: (151565, 32) Val shape: (37892, 32)
Positive class in train: 0.06929040345726256
Positive class in val:   0.06930222738308878
After oversampling: (162222, 32)
Positive class proportion: 0.13043237045530198
